In [2]:
import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [24]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [51]:
test.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4,4,4,1,...,2,3,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4,4,4,2,...,1,3,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4,4,4,2,...,2,3,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4,4,4,2,...,2,3,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4,4,4,1,...,5,3,0,0,0,0,0,0,0,0


In [150]:
test_attr = test.ix[:, 2:]
test_attr.head()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,Niğde,Other,FC,1,4,4,4,1,2,5,...,2,3,0,0,0,0,0,0,0,0
1,Konya,Other,IL,3,4,4,4,2,2,5,...,1,3,0,0,0,0,0,0,0,0
2,Ankara,Big Cities,FC,3,4,4,4,2,2,5,...,2,3,0,0,0,0,0,0,0,0
3,Kocaeli,Other,IL,2,4,4,4,2,3,5,...,2,3,0,4,0,0,0,0,0,0
4,Afyonkarahisar,Other,FC,2,4,4,4,1,2,5,...,5,3,0,0,0,0,0,0,0,0


In [151]:
attr = train.ix[:, 2:-1]
attr.head()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,İstanbul,Big Cities,IL,4,5.0,4,4,2,2,5,...,2.0,3.0,5,3,4,5,5,4,3,4
1,Ankara,Big Cities,FC,4,5.0,4,4,1,2,5,...,3.0,3.0,0,0,0,0,0,0,0,0
2,Diyarbakır,Other,IL,2,4.0,2,5,2,3,5,...,1.0,3.0,0,0,0,0,0,0,0,0
3,Tokat,Other,IL,6,4.5,6,6,4,4,10,...,2.5,7.5,25,12,10,6,18,12,12,6
4,Gaziantep,Other,IL,3,4.0,3,4,2,2,5,...,1.0,3.0,5,1,3,2,3,4,3,3


In [92]:
target = train.ix[:, -1].values

In [154]:
def add_date_features(data, date_col):
    
    dates = pd.to_datetime(date_col)
    data['Year'] = dates.dt.year
    data['Month'] = dates.dt.month
    data['Week'] = dates.dt.weekofyear
    
    return data

attr = add_date_features(attr, train['Open Date'])
test_attr = add_date_features(test_attr, test['Open Date'])

In [156]:
from sklearn.preprocessing import LabelEncoder

enc_cols = ['City', 'City Group', 'Type']
combined_data = attr.append(test_attr)

def gen_encoders(data, cols):
    
    les = {}
    
    for col in cols:
        le = LabelEncoder()
        le.fit(data[col].values)
        les[col] = le
    return les

all_les = gen_encoders(combined_data, enc_cols)

In [158]:
def encode(data, encoders):
    for k, v in encoders.iteritems():
        data[k] = v.transform(data[k])
        
    return data
        
attr = encode(attr, all_les)
test_attr = encode(test_attr, all_les)

In [159]:
new_reg = ExtraTreesRegressor()
new_reg.fit(attr, target)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)

In [160]:
results = new_reg.predict(test_attr)

In [161]:
output_df = pd.DataFrame({'Prediction': results})

In [162]:
output_df.to_csv('submission.csv')